In [ ]:
import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('ibtracs.ALL.list.v04r01 (1).csv', low_memory=False)

<ipython-input-3-27e142b34f24>:1: DtypeWarning: Columns (1,2,8,9,14,19,20,172,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('ibtracs.ALL.list.v04r01 (1).csv')


In [ ]:
def preprocess_cyclone_data(df, task='path'):
    """
    Preprocesses cyclone data for LSTM and ML models.

    Parameters:
        df (DataFrame): Raw cyclone dataset.
        task (str): 'path' for trajectory prediction (LSTM), 'speed_dir' for speed/direction prediction (ML).

    Returns:
        Processed DataFrame (X, y) and scalers (if LSTM).
    """
    df = df.copy()  # Avoid modifying original data

    # Converting time column to datetime
    df['ISO_TIME'] = pd.to_datetime(df['ISO_TIME'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    # Setting index for resampling
    df.set_index('ISO_TIME', inplace=True)

    # Dealing with duplicate timestamps (keep first occurrence)
    df = df[~df.index.duplicated(keep='first')]

    # Converting to numeric columns before resampling
    for col in ['STORM_SPEED', 'STORM_DIR', 'LAT', 'LON']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Converting objects to numeric before interpolating
    df = df.infer_objects(copy=False)

    # data will be resampled every 6 hours and interpolate missing values
    df = df.resample('6h').interpolate()

    # Reset index after resampling
    df.reset_index(inplace=True)

    # Circular encoding for wind direction
    df['dir_sin'] = np.sin(np.deg2rad(df['STORM_DIR']))
    df['dir_cos'] = np.cos(np.deg2rad(df['STORM_DIR']))

    # Interaction terms using available parameters
    df['lat_lon_interaction'] = df['LAT'] * df['LON']
    df['speed_lat_interaction'] = df['STORM_SPEED'] * df['LAT']
    df['speed_lon_interaction'] = df['STORM_SPEED'] * df['LON']

    # some lag features
    df['STORM_SPEED_LAG1'] = df['STORM_SPEED'].shift(1)
    df['LAT_LAG'] = df['LAT'].shift(1)
    df['LON_LAG'] = df['LON'].shift(1)

    # creating moving Averages
    df['LAT_MA3'] = df['LAT'].rolling(window=3).mean()
    df['LON_MA3'] = df['LON'].rolling(window=3).mean()
    df['SPEED_MA3'] = df['STORM_SPEED'].rolling(window=3).mean()

    # Deriving dist change and speed change
    df['DIST_CHANGE'] = np.sqrt((df['LAT'] - df['LAT_LAG'])**2 + (df['LON'] - df['LON_LAG'])**2)
    df['SPEED_CHANGE'] = df['STORM_SPEED'].diff()

    # Standard deviation over 3 periods
    df['SPEED_STD3'] = df['STORM_SPEED'].rolling(window=3).std()

    # mapping time-based features
    df['YEAR'] = df['ISO_TIME'].dt.year
    df['MONTH'] = df['ISO_TIME'].dt.month
    df['DAY'] = df['ISO_TIME'].dt.day
    df['HOUR'] = df['ISO_TIME'].dt.hour

    # Filling missing values with forward fill
    df.ffill(inplace=True)

    # Defining feature sets based on task
    if task == 'path':
        features = ['LAT', 'LON', 'STORM_SPEED', 'HOUR', 'MONTH',
                    'lat_lon_interaction', 'speed_lat_interaction', 'speed_lon_interaction',
                    'dir_sin', 'dir_cos']
        target_cols = ['LAT', 'LON']  # Predicting future position
    elif task == 'speed_dir':
        features = ['LAT', 'LON', 'HOUR', 'MONTH', 'dir_sin', 'dir_cos',
                    'STORM_SPEED_LAG1', 'LAT_LAG', 'LON_LAG',
                    'LAT_MA3', 'LON_MA3', 'SPEED_MA3', 'SPEED_STD3']
        target_cols = ['STORM_SPEED', 'STORM_DIR']  # Predicting speed and direction
    else:
        raise ValueError("Invalid task type! Choose 'path' or 'speed_dir'.")

    # Dropping NaN values after feature creation
    df.dropna(inplace=True)

    # Extracting features and target
    X = df[features].values
    y = df[target_cols].values

    # Scaling (only for LSTM )
    if task == 'path':
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X = scaler_X.fit_transform(X)
        y = scaler_y.fit_transform(y)

        # got to reshape for LSTM [samples, timesteps, features]
        X = X.reshape((X.shape[0], 1, X.shape[1]))

        return X, y, scaler_X, scaler_y  # Returning scalers for inverse transformation

    return X, y  # our ML models don’t need reshaping or scaling

In [ ]:
# Preprocess the data using previously built function
X, y = preprocess_cyclone_data(df, task='speed_dir')  # X = features, y = [STORM_SPEED, STORM_DIR]

# Splitting into training & test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extracting target variables
y_train_speed, y_train_dir = y_train[:, 0], y_train[:, 1]  # Speed & Direction separately
y_test_speed, y_test_dir = y_test[:, 0], y_test[:, 1]

# making XGBoost models
speed_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
dir_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Training the models
speed_model.fit(X_train, y_train_speed)
dir_model.fit(X_train, y_train_dir)

# Predicting on test data
y_pred_speed = speed_model.predict(X_test)
y_pred_dir = dir_model.predict(X_test)

# Lets evaluate performance
print("Speed Model Performance")
print("MAE:", mean_absolute_error(y_test_speed, y_pred_speed))
print("R² Score:", r2_score(y_test_speed, y_pred_speed))

print("\n Direction Model Performance**")
print("MAE:", mean_absolute_error(y_test_dir, y_pred_dir))
print("R² Score:", r2_score(y_test_dir, y_pred_dir))

<ipython-input-12-5f94d251fc41>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')
<ipython-input-12-5f94d251fc41>:35: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample('6h').interpolate()


⚡ **Speed Model Performance**
MAE: 0.9540082789638384
R² Score: 0.93415075550222

🧭 **Direction Model Performance**
MAE: 0.10304005607331024
R² Score: 0.9999960459225579


In [ ]:
import joblib


joblib.dump(speed_model, "speed_model.pkl")  # Saving speed prediction model
joblib.dump(dir_model, "dir_model.pkl")  # Saving direction prediction model

print("✅ Models saved successfully!")


✅ Models saved successfully!


In [ ]:
def preprocess_cyclone_data_lstm(df, scaler_X=None, scaler_y=None, training=True):
    """Preprocess cyclone data for LSTM training and inference."""

    df = df.copy()  # Avoid modifying the original DataFrame
    df['ISO_TIME'] = pd.to_datetime(df['ISO_TIME'], errors='coerce')

    # Extract time-based features
    df['hour'] = df['ISO_TIME'].dt.hour.fillna(0).astype(int)
    df['month'] = df['ISO_TIME'].dt.month.fillna(0).astype(int)

    # Convert numeric columns to float
    numeric_cols = ['LAT', 'LON', 'STORM_SPEED', 'STORM_DIR']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Filling missing values with median instead of dropping data
    df.fillna(df.median(numeric_only=True), inplace=True)

    # Convert storm direction to sine & cosine
    df['dir_sin'] = np.sin(np.deg2rad(df['STORM_DIR']))
    df['dir_cos'] = np.cos(np.deg2rad(df['STORM_DIR']))

    # Interaction terms
    df['lat_lon_interaction'] = df['LAT'] * df['LON']
    df['speed_lat_interaction'] = df['STORM_SPEED'] * df['LAT']
    df['speed_lon_interaction'] = df['STORM_SPEED'] * df['LON']

    # feature columns
    features = [
        'LAT', 'LON', 'STORM_SPEED', 'hour', 'month',
        'lat_lon_interaction', 'speed_lat_interaction', 'speed_lon_interaction',
        'dir_sin', 'dir_cos'
    ]
    target_cols = ['LAT', 'LON']

    # Extract feature values
    X = df[features].values
    y = df[target_cols].values if training else None

    # Load or fit scalers
    if training:
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        X_scaled = scaler_X.fit_transform(X)
        y_scaled = scaler_y.fit_transform(y)

        # Save scalers for later inference
        joblib.dump(scaler_X, "scaler_X.pkl")
        joblib.dump(scaler_y, "scaler_y.pkl")

    else:
        scaler_X = joblib.load("scaler_X.pkl")
        X_scaled = scaler_X.transform(X)
        y_scaled = None

    # Reshape for LSTM input (samples, timesteps=1, features)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

    return X_scaled, y_scaled, scaler_X, scaler_y

# Load dataset (Ensure `df` is available)
X_scaled, y_scaled, scaler_X, scaler_y = preprocess_cyclone_data_lstm(df)

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Build LSTM Model
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(2)  # Predicting (LAT, LON)
])

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Define Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train Model
history = model.fit(
    X_train, y_train,
    epochs=50, batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1, callbacks=[early_stopping]
)

# Save trained model for later inference
model.save("cyclone_lstm_model.h5")

# Predict
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Display Sample Predictions
y_test_actual = scaler_y.inverse_transform(y_test)
for i in range(5):
    print(f"Actual: {y_test_actual[i]}, Predicted: {y_pred[i]}")

# Evaluate on test set (original scale)
lat_mae = mean_absolute_error(y_test_actual[:, 0], y_pred[:, 0])
lon_mae = mean_absolute_error(y_test_actual[:, 1], y_pred[:, 1])
lat_rmse = np.sqrt(mean_squared_error(y_test_actual[:, 0], y_pred[:, 0]))
lon_rmse = np.sqrt(mean_squared_error(y_test_actual[:, 1], y_pred[:, 1]))
lat_r2 = r2_score(y_test_actual[:, 0], y_pred[:, 0])
lon_r2 = r2_score(y_test_actual[:, 1], y_pred[:, 1])
overall_r2 = r2_score(y_test_actual, y_pred, multioutput='variance_weighted')
overall_rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred))

print("\nLSTM Path Model Performance")
print(f"LAT -> MAE: {lat_mae:.4f}, RMSE: {lat_rmse:.4f}, R²: {lat_r2:.4f}")
print(f"LON -> MAE: {lon_mae:.4f}, RMSE: {lon_rmse:.4f}, R²: {lon_r2:.4f}")
print(f"Overall -> RMSE: {overall_rmse:.4f}, R²: {overall_r2:.4f}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2527/2527 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - loss: 0.0727 - mae: 0.1120 - val_loss: 1.0720e-04 - val_mae: 0.0076
Epoch 2/50
2527/2527 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 8.4055e-05 - mae: 0.0066 - val_loss: 6.6684e-05 - val_mae: 0.0065
Epoch 3/50
2527/2527 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 5.0780e-05 - mae: 0.0052 - val_loss: 8.8522e-05 - val_mae: 0.0071
Epoch 4/50
2527/2527 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 4.1799e-05 - mae: 0.0046 - val_loss: 3.0547e-05 - val_mae: 0.0039
Epoch 5/50
2527/2527 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 3.0355e-05 - mae: 0.0040 - val_loss: 2.0953e-05 - val_mae: 0.0038
Epoch 6/50
2527/2527 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - loss: 2.2312e-05 - mae: 0.0035 - val_loss: 8.9374e-06 - val_mae: 0.0023
Epoch 7/50
2527/2527 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - loss: 2.0813e-05 - mae: 0.0032 - val_loss: 2.0565e-05 - val_mae: 0.0034
Epoch 8/50
2527/2527 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 2.0385e-05 - mae: 0.0032 - val_loss: 2.5539e-05 -

632/632 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Actual: [17.6 87.2], Predicted: [17.537634 86.9107  ]
Actual: [18.2 84.6], Predicted: [18.213432 84.32055 ]
Actual: [ 11.5 111.7], Predicted: [ 11.43135 111.57744]
Actual: [ 25.  -60.4], Predicted: [ 24.967434 -60.16016 ]
Actual: [-14.8  76. ], Predicted: [-14.761486  75.89571 ]


In [ ]:
# Load trained model and scalers
model = load_model("cyclone_lstm_model.h5")
scaler_X = joblib.load("scaler_X.pkl")
scaler_y = joblib.load("scaler_y.pkl")

# Load new cyclone data (`new_df` should be a Pandas DataFrame)
X_new, _, _, _ = preprocess_cyclone_data_lstm(new_df, training=False)

# Predict
y_pred_scaled = model.predict(X_new)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Print predicted locations
print("Predicted cyclone locations (LAT, LON):", y_pred)

NameError: name 'load_model' is not defined